# HumanEval Evaluation: Error-Aware Prompt (Detailed)

This notebook evaluates `deepseek-ai/deepseek-coder-6.7b-instruct` on HumanEval using an **error-aware prompt** designed to address common syntax and runtime errors.

**Purpose:** Improve Pass@1 by explicitly instructing the model to avoid common errors:
- Use proper `->` syntax for return type annotations
- Close all parentheses/brackets
- Output only code, no explanations
- Use exact function/variable names

**Baseline:** Original prompt achieved 61.59% Pass@1

## 1. Setup & Dependencies

In [1]:
# Install required packages
!pip install -q transformers accelerate bitsandbytes torch
!pip install -q human-eval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour 

In [2]:
import json
import os
import re
import ast
import traceback
from typing import Dict, List, Tuple, Any
from collections import defaultdict
import signal
from contextlib import contextmanager

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from human_eval.data import read_problems, write_jsonl

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

PyTorch version: 2.6.0+cu124
CUDA available: True
GPU: Tesla T4


## 2. Load DeepSeek Coder Model

In [3]:
MODEL_NAME = "deepseek-ai/deepseek-coder-6.7b-instruct"

# 4-bit quantization config for memory efficiency
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

print(f"Loading model: {MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True
)
model.eval()
print("Model loaded successfully!")

Loading model: deepseek-ai/deepseek-coder-6.7b-instruct


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

Model loaded successfully!


## 3. Load HumanEval Dataset

In [4]:
# Load HumanEval problems
problems = read_problems()
print(f"Loaded {len(problems)} HumanEval problems")

# Display sample problem
sample_id = "HumanEval/0"
print(f"\nSample Problem ({sample_id}):")
print("="*50)
print(problems[sample_id]["prompt"][:500])

Loaded 164 HumanEval problems

Sample Problem (HumanEval/0):
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """



## 4. Error-Aware Prompt (Detailed Version)

This prompt explicitly addresses the common error patterns identified in the baseline evaluation:
- **31.1% Syntax Errors**: Missing `->`, unclosed brackets, mixed explanations
- **9.2% Runtime Errors**: Typos in function names

In [5]:
def create_error_aware_prompt(problem_prompt: str) -> str:
    """
    Create an ERROR-AWARE prompt with explicit instructions to avoid common errors.
    """
    instruction = f"""Complete the following Python function.

CRITICAL REQUIREMENTS:
1. Use proper Python syntax - always use '-> ReturnType' (not '- ReturnType') for return type annotations
2. Output ONLY the function body code, no explanations or test code after
3. Ensure all parentheses, brackets, and quotes are properly closed
4. Use exact function/variable names as given - no typos
5. Return proper types matching the function signature

{problem_prompt}"""
    
    # DeepSeek Coder instruction format
    formatted_prompt = f"""### Instruction:
{instruction}

### Response:
"""
    return formatted_prompt


def generate_completion(prompt: str, max_new_tokens: int = 512) -> str:
    """
    Generate code completion using DeepSeek Coder.
    """
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,  # Greedy decoding for pass@1
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            temperature=None,
            top_p=None,
        )
    
    # Decode only the generated part
    generated_tokens = outputs[0][inputs['input_ids'].shape[1]:]
    completion = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    
    return completion


def extract_function_body(completion: str, prompt: str) -> str:
    """
    Extract and clean the function body from the completion.
    Enhanced extraction to handle common model output issues.
    """
    # Remove markdown code blocks if present
    completion = re.sub(r'```python\n?', '', completion)
    completion = re.sub(r'```\n?', '', completion)
    
    # Stop at common end markers
    stop_sequences = ["\n\n\n", "\n###", "\nif __name__", "\n# Test", "\n# Example", "\nThis function", "\nThis solution", "\nPlease note"]
    for stop in stop_sequences:
        if stop in completion:
            completion = completion[:completion.index(stop)]
    
    return completion.strip()


print("Error-aware prompt generation functions defined.")
print("\n--- Sample Error-Aware Prompt ---")
sample_prompt = create_error_aware_prompt(problems["HumanEval/0"]["prompt"])
print(sample_prompt[:800])

Error-aware prompt generation functions defined.

--- Sample Error-Aware Prompt ---
### Instruction:
Complete the following Python function.

CRITICAL REQUIREMENTS:
1. Use proper Python syntax - always use '-> ReturnType' (not '- ReturnType') for return type annotations
2. Output ONLY the function body code, no explanations or test code after
3. Ensure all parentheses, brackets, and quotes are properly closed
4. Use exact function/variable names as given - no typos
5. Return proper types matching the function signature

from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """


### Resp


## 5. Execution & Error Classification

In [6]:
class TimeoutException(Exception):
    pass


@contextmanager
def time_limit(seconds: int):
    """
    Context manager to limit execution time.
    """
    def signal_handler(signum, frame):
        raise TimeoutException("Execution timed out!")
    
    old_handler = signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    
    try:
        yield
    finally:
        signal.alarm(0)
        signal.signal(signal.SIGALRM, old_handler)


def classify_error(code: str, test_code: str, entry_point: str) -> Tuple[str, str]:
    """
    Execute code and classify any errors.
    """
    # First check for syntax errors
    try:
        ast.parse(code)
    except SyntaxError as e:
        return 'syntax_error', str(e)
    
    # Check if the entry point function exists in code
    if f"def {entry_point}" not in code:
        return 'incomplete_code', f"Function '{entry_point}' not found in generated code"
    
    # Try to execute with tests
    full_code = code + "\n" + test_code
    
    try:
        ast.parse(full_code)
    except SyntaxError as e:
        return 'syntax_error', str(e)
    
    # Execute with timeout
    try:
        exec_globals = {}
        with time_limit(5):
            exec(full_code, exec_globals)
        return 'pass', ''
    except TimeoutException:
        return 'timeout', 'Execution timed out (>5 seconds)'
    except AssertionError as e:
        return 'runtime_error', f"Assertion failed: {str(e)}"
    except Exception as e:
        error_type = type(e).__name__
        return 'runtime_error', f"{error_type}: {str(e)}"


print("Execution and error classification functions defined.")

Execution and error classification functions defined.


## 6. Run Evaluation on All 164 Problems

In [7]:
# Initialize tracking variables
results = []
error_counts = defaultdict(int)
successful_problems = []
failed_problems = defaultdict(list)

# Output log file
output_log_path = "output_log_error_aware.txt"

print(f"Starting ERROR-AWARE PROMPT evaluation on {len(problems)} problems...")
print("="*60)

with open(output_log_path, 'w', encoding='utf-8') as log_file:
    log_file.write("HumanEval Evaluation Log - ERROR-AWARE PROMPT (DETAILED)\n")
    log_file.write("Model: deepseek-ai/deepseek-coder-6.7b-instruct\n")
    log_file.write("="*60 + "\n\n")
    
    for i, (task_id, problem) in enumerate(problems.items()):
        problem_num = int(task_id.split("/")[1])
        prompt = problem["prompt"]
        test_code = problem["test"]
        entry_point = problem["entry_point"]
        
        # Create ERROR-AWARE prompt and generate completion
        formatted_prompt = create_error_aware_prompt(prompt)
        completion = generate_completion(formatted_prompt)
        cleaned_completion = extract_function_body(completion, prompt)
        
        # Combine prompt with completion for full code
        full_code = prompt + cleaned_completion
        
        # Classify result
        status, error_msg = classify_error(full_code, test_code, entry_point)
        
        # Track results
        error_counts[status] += 1
        
        if status == 'pass':
            successful_problems.append(problem_num)
        else:
            failed_problems[status].append(problem_num)
        
        results.append({
            "task_id": task_id,
            "problem_num": problem_num,
            "status": status,
            "error": error_msg,
            "completion": cleaned_completion,
            "prompt_type": "error_aware_detailed"
        })
        
        # Write to log file
        log_file.write(f"{'='*60}\n")
        log_file.write(f"PROBLEM {problem_num}: {task_id}\n")
        log_file.write(f"STATUS: {status.upper()}\n")
        log_file.write(f"PROMPT TYPE: ERROR-AWARE (DETAILED)\n")
        log_file.write(f"{'='*60}\n\n")
        log_file.write(f"--- ORIGINAL PROMPT ---\n")
        log_file.write(f"{prompt}\n\n")
        log_file.write(f"--- FORMATTED PROMPT TO MODEL ---\n")
        log_file.write(f"{formatted_prompt}\n\n")
        log_file.write(f"--- RAW MODEL OUTPUT ---\n")
        log_file.write(f"{completion}\n\n")
        log_file.write(f"--- CLEANED COMPLETION ---\n")
        log_file.write(f"{cleaned_completion}\n\n")
        log_file.write(f"--- FULL GENERATED CODE ---\n")
        log_file.write(f"{full_code}\n\n")
        if error_msg:
            log_file.write(f"--- ERROR MESSAGE ---\n")
            log_file.write(f"{error_msg}\n\n")
        log_file.write("\n")
        
        # Progress update
        if (i + 1) % 10 == 0 or (i + 1) == len(problems):
            print(f"Progress: {i+1}/{len(problems)} | Current pass rate: {error_counts['pass']/(i+1)*100:.1f}%")

print("\nEvaluation complete!")
print(f"Output log saved to: {output_log_path}")

Starting ERROR-AWARE PROMPT evaluation on 164 problems...
Progress: 10/164 | Current pass rate: 20.0%
Progress: 20/164 | Current pass rate: 30.0%
Progress: 30/164 | Current pass rate: 33.3%
Progress: 40/164 | Current pass rate: 50.0%
Progress: 50/164 | Current pass rate: 60.0%
Progress: 60/164 | Current pass rate: 65.0%
Progress: 70/164 | Current pass rate: 68.6%
Progress: 80/164 | Current pass rate: 71.2%
['A+', 'B', 'C-', 'C', 'A-']
Progress: 90/164 | Current pass rate: 74.4%
Progress: 100/164 | Current pass rate: 77.0%
Progress: 110/164 | Current pass rate: 78.2%
Progress: 120/164 | Current pass rate: 80.0%
Progress: 130/164 | Current pass rate: 81.5%
Progress: 140/164 | Current pass rate: 82.9%
Progress: 150/164 | Current pass rate: 83.3%
Progress: 160/164 | Current pass rate: 84.4%
Progress: 164/164 | Current pass rate: 84.8%

Evaluation complete!
Output log saved to: output_log_error_aware.txt


## 7. Calculate & Display Results

In [8]:
# Calculate metrics
total_problems = len(problems)
passed = error_counts['pass']
syntax_errors = error_counts['syntax_error']
runtime_errors = error_counts['runtime_error']
incomplete_errors = error_counts['incomplete_code']
timeout_errors = error_counts['timeout']

pass_at_1 = (passed / total_problems) * 100
syntax_error_pct = (syntax_errors / total_problems) * 100
runtime_error_pct = (runtime_errors / total_problems) * 100
incomplete_error_pct = (incomplete_errors / total_problems) * 100
timeout_error_pct = (timeout_errors / total_problems) * 100

# Baseline comparison
baseline_pass_at_1 = 61.59
improvement = pass_at_1 - baseline_pass_at_1

print("\n" + "="*60)
print("HUMANEVAL EVALUATION RESULTS - ERROR-AWARE PROMPT (DETAILED)")
print("Model: deepseek-ai/deepseek-coder-6.7b-instruct")
print("="*60)

print(f"\ PASS@1 METRIC:")
print(f"   Pass@1: {pass_at_1:.2f}% ({passed}/{total_problems})")
print(f"   Baseline: {baseline_pass_at_1:.2f}%")
print(f"   Improvement: {improvement:+.2f}%")

print(f"\n ERROR BREAKDOWN (on {total_problems} problems):")
print(f"    Passed:           {passed:3d} ({pass_at_1:.2f}%)")
print(f"    Syntax Errors:    {syntax_errors:3d} ({syntax_error_pct:.2f}%)")
print(f"    Runtime Errors:   {runtime_errors:3d} ({runtime_error_pct:.2f}%)")
print(f"    Incomplete Code:  {incomplete_errors:3d} ({incomplete_error_pct:.2f}%)")
print(f"    Timeouts:         {timeout_errors:3d} ({timeout_error_pct:.2f}%)")


HUMANEVAL EVALUATION RESULTS - ERROR-AWARE PROMPT (DETAILED)
Model: deepseek-ai/deepseek-coder-6.7b-instruct
\ PASS@1 METRIC:
   Pass@1: 84.76% (139/164)
   Baseline: 61.59%
   Improvement: +23.17%

 ERROR BREAKDOWN (on 164 problems):
    Passed:           139 (84.76%)
    Syntax Errors:     21 (12.80%)
    Runtime Errors:     4 (2.44%)
    Incomplete Code:    0 (0.00%)
    Timeouts:           0 (0.00%)


In [9]:
# Display successful problem numbers
print("\n" + "="*60)
print("SUCCESSFULLY EXECUTED PROBLEMS")
print("="*60)

successful_problems_sorted = sorted(successful_problems)
print(f"\nTotal: {len(successful_problems_sorted)} problems passed")
print(f"\nProblem Numbers: {successful_problems_sorted}")


SUCCESSFULLY EXECUTED PROBLEMS

Total: 139 problems passed

Problem Numbers: [1, 6, 10, 13, 16, 18, 20, 23, 24, 25, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 65, 66, 67, 68, 69, 70, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163]


In [10]:
# Display failed problems by category
print("\n" + "="*60)
print("FAILED PROBLEMS BY CATEGORY")
print("="*60)

if failed_problems['syntax_error']:
    print(f"\n Syntax Errors ({len(failed_problems['syntax_error'])}):")
    print(f"   {sorted(failed_problems['syntax_error'])}")

if failed_problems['runtime_error']:
    print(f"\n Runtime Errors ({len(failed_problems['runtime_error'])}):")
    print(f"   {sorted(failed_problems['runtime_error'])}")

if failed_problems['incomplete_code']:
    print(f"\n Incomplete Code ({len(failed_problems['incomplete_code'])}):")
    print(f"   {sorted(failed_problems['incomplete_code'])}")

if failed_problems['timeout']:
    print(f"\n  Timeouts ({len(failed_problems['timeout'])}):")
    print(f"   {sorted(failed_problems['timeout'])}")


FAILED PROBLEMS BY CATEGORY

 Syntax Errors (21):
   [0, 2, 3, 4, 5, 8, 9, 11, 12, 14, 15, 17, 19, 21, 26, 27, 52, 64, 71, 102, 141]

 Runtime Errors (4):
   [7, 22, 28, 29]


In [11]:
# Save summary
summary_path = "evaluation_summary_error_aware.txt"

with open(summary_path, 'w') as f:
    f.write("HUMANEVAL EVALUATION SUMMARY - ERROR-AWARE PROMPT (DETAILED)\n")
    f.write("Model: deepseek-ai/deepseek-coder-6.7b-instruct\n")
    f.write("="*60 + "\n\n")
    
    f.write(f"PASS@1: {pass_at_1:.2f}% ({passed}/{total_problems})\n")
    f.write(f"BASELINE: {baseline_pass_at_1:.2f}%\n")
    f.write(f"IMPROVEMENT: {improvement:+.2f}%\n\n")
    
    f.write("ERROR BREAKDOWN:\n")
    f.write(f"  - Passed:          {passed} ({pass_at_1:.2f}%)\n")
    f.write(f"  - Syntax Errors:   {syntax_errors} ({syntax_error_pct:.2f}%)\n")
    f.write(f"  - Runtime Errors:  {runtime_errors} ({runtime_error_pct:.2f}%)\n")
    f.write(f"  - Incomplete Code: {incomplete_errors} ({incomplete_error_pct:.2f}%)\n")
    f.write(f"  - Timeouts:        {timeout_errors} ({timeout_error_pct:.2f}%)\n\n")
    
    f.write("SUCCESSFUL PROBLEM NUMBERS:\n")
    f.write(f"{successful_problems_sorted}\n\n")
    
    f.write("FAILED PROBLEMS BY CATEGORY:\n")
    if failed_problems['syntax_error']:
        f.write(f"  Syntax Errors: {sorted(failed_problems['syntax_error'])}\n")
    if failed_problems['runtime_error']:
        f.write(f"  Runtime Errors: {sorted(failed_problems['runtime_error'])}\n")
    if failed_problems['incomplete_code']:
        f.write(f"  Incomplete Code: {sorted(failed_problems['incomplete_code'])}\n")
    if failed_problems['timeout']:
        f.write(f"  Timeouts: {sorted(failed_problems['timeout'])}\n")

print(f"\n Summary saved to: {summary_path}")


 Summary saved to: evaluation_summary_error_aware.txt


In [12]:
# Save detailed results as JSON
results_path = "evaluation_results_error_aware.json"

detailed_results = {
    "model": MODEL_NAME,
    "prompt_type": "error_aware_detailed",
    "total_problems": total_problems,
    "pass_at_1": pass_at_1,
    "baseline_pass_at_1": baseline_pass_at_1,
    "improvement": improvement,
    "error_counts": dict(error_counts),
    "error_percentages": {
        "pass": pass_at_1,
        "syntax_error": syntax_error_pct,
        "runtime_error": runtime_error_pct,
        "incomplete_code": incomplete_error_pct,
        "timeout": timeout_error_pct
    },
    "successful_problems": successful_problems_sorted,
    "failed_problems": {k: sorted(v) for k, v in failed_problems.items()},
    "detailed_results": results
}

with open(results_path, 'w') as f:
    json.dump(detailed_results, f, indent=2)

print(f" Detailed results saved to: {results_path}")

 Detailed results saved to: evaluation_results_error_aware.json


## 8. Final Summary & Comparison

In [13]:
print("\nEVALUATION COMPLETE!")
print("="*30)

print(f"""
ERROR-AWARE PROMPT (DETAILED) RESULTS
-------------------------------------
Model:       deepseek-ai/deepseek-coder-6.7b-instruct
Benchmark:   HumanEval ({total_problems} problems)
Prompt Type: ERROR-AWARE (DETAILED)

PASS@1:      {pass_at_1:.2f}%
BASELINE:    {baseline_pass_at_1:.2f}%
IMPROVEMENT: {improvement:+.2f}%

Error Breakdown:
  Passed:          {passed} problems ({pass_at_1:.2f}%)
  Syntax Errors:   {syntax_errors} problems ({syntax_error_pct:.2f}%)
  Runtime Errors:  {runtime_errors} problems ({runtime_error_pct:.2f}%)
  Incomplete Code: {incomplete_errors} problems ({incomplete_error_pct:.2f}%)
  Timeouts:        {timeout_errors} problems ({timeout_error_pct:.2f}%)

Output Files:
  output_log_error_aware.txt
  evaluation_summary_error_aware.txt
  evaluation_results_error_aware.json
""")


EVALUATION COMPLETE!

ERROR-AWARE PROMPT (DETAILED) RESULTS
-------------------------------------
Model:       deepseek-ai/deepseek-coder-6.7b-instruct
Benchmark:   HumanEval (164 problems)
Prompt Type: ERROR-AWARE (DETAILED)

PASS@1:      84.76%
BASELINE:    61.59%
IMPROVEMENT: +23.17%

Error Breakdown:
  Passed:          139 problems (84.76%)
  Syntax Errors:   21 problems (12.80%)
  Runtime Errors:  4 problems (2.44%)
  Incomplete Code: 0 problems (0.00%)
  Timeouts:        0 problems (0.00%)

Output Files:
  output_log_error_aware.txt
  evaluation_summary_error_aware.txt
  evaluation_results_error_aware.json

